In [ ]:
import orbipy as op, numpy as np, pandas as pd
import matplotlib.pyplot as plt

model = op.crtbp3_model('Earth-Moon (default)')
plot_creator = op.plotter.from_model(model, length_units='Mm')
scaler = op.scaler.from_model(model)

In [ ]:
disturbance = 1e-6
model.integrator.set_params(max_step=scaler(1, 'h-nd'))

In [ ]:
path_report = ''
data = pd.read_csv(path_report, dtype='float64', low_memory=False)

In [ ]:
data.info()

In [ ]:
data['norm'] = np.linalg.norm(data[['vxc', 'vyc', 'vzc']], axis=1)
optimal = data.sort_values(by=['norm']).iloc[0]

In [ ]:
print('Optimal velocity correction:', scaler(optimal.values[-1], 'nd/nd-m/s'), 'm/s')

In [ ]:
top = data[data['norm'] < scaler(20., 'm/s-nd/nd')]
len(top.index)

In [ ]:
unstable_state, unstable_time, stable_state, stable_time, correction = \
    optimal[: 6], optimal[6], optimal[7: 13], optimal[13], optimal[14: -1]

In [ ]:
arc_data = model.prop(stable_state, 0., stable_time)
check_state = arc_data.iloc[-1].values[1: ]

check_state[3] -= disturbance
detector = op.event_detector(model, [op.eventX(1 - model.mu, terminal=True, count=1)])
constrained_data, event_data = detector.prop(check_state, 0., -6 * np.pi, last_state='last')

plot_creator.plot_proj(constrained_data, projections=('x-y',), centers={'x': 1 - model.mu}, linewidth=0.7);

In [ ]:
unconstrained_time = abs(constrained_data.iloc[-1].values[0])
tolerance_updated_state = constrained_data.iloc[-1].values[1: ]

In [ ]:
complete_time = 0

arc_data = model.prop(unstable_state, 0., unstable_time)
inter_state = arc_data.iloc[-1].values[1: ]
complete_time += unstable_time

inter_state[3] -= disturbance
detector = op.event_detector(model, [op.eventX(1 - model.mu, terminal=True)])
constrained_data, event_data = detector.prop(inter_state, unstable_time, 6 * np.pi, last_state='last')
complete_time += event_data.iloc[-1].values[3]

until_surface_data = pd.concat([arc_data, constrained_data], ignore_index=True)
unconstrained_data = model.prop(tolerance_updated_state, 0., unconstrained_time)
complete_time += unconstrained_time

after_surface_data = pd.concat([until_surface_data, unconstrained_data], ignore_index=True)

In [ ]:
axes = plot_creator.plot_proj(after_surface_data, projections=('x-y',), centers={'x': 1 - model.mu}, 
                              fsize=(12, 8), linewidth=0.7);
plot_creator.plot_proj(projections=('x-y',), plottables=[plot_creator.L1, plot_creator.m, plot_creator.L2], 
                       centers={'x': 1 - model.mu}, colors='k', marker='o', ax=axes);

In [ ]:
path_continue_transit = ''
transit_data = pd.read_csv(path_continue_transit)

transit_data['t'] += complete_time
transit_data = transit_data.reindex(index=transit_data.index[: : -1]).reset_index(drop=True)

complete_data = pd.concat([after_surface_data, transit_data], ignore_index=True)

In [ ]:
axes = plot_creator.plot_proj(complete_data, projections=('x-y',), centers={'x': 1 - model.mu}, 
                              fsize=(12, 8), linewidth=0.7);
plot_creator.plot_proj(projections=('x-y',), plottables=[plot_creator.L1, plot_creator.m, plot_creator.L2], 
                       centers={'x': 1 - model.mu}, colors='k', marker='o', ax=axes);
plt.savefig('transit.png')